In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical

2024-03-07 19:07:12.486058: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 19:07:12.486210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 19:07:12.650810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
images = []
X = [] #copy forged images
Y = [] #their respective masks

In [3]:
def load_images_from_directory(directory_path):
    images = []

    files = sorted(os.listdir(directory_path))
    i = 1

    for file_name in files:
        file_path = os.path.join(directory_path, file_name)

        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            if i % 1000 == 0:
                print(f'loaded {i} images')
            im = Image.open(file_path)
            images.append(np.array(im))  # Convert the image to a NumPy array
            i += 1

    return images

In [4]:
directory_path = "/kaggle/input/copymovemasks/CoMoFoD_small_v2"
images = load_images_from_directory(directory_path)

loaded 1000 images
loaded 2000 images
loaded 3000 images
loaded 4000 images
loaded 5000 images
loaded 6000 images
loaded 7000 images
loaded 8000 images
loaded 9000 images
loaded 10000 images


In [5]:
print(len(images))

10400


In [6]:
matrix = []
num_rows = len(images)//26

for i in range(0, len(images), 26):
    # Take a mask image followed by 25 corresponding images and create a row in the matrix
    row = images[i:i + 26]
    matrix.append(row)
    
for i in range(num_rows):
    row = matrix[i]
    
    # For x, take images from index 1 to 25 in each row
    X.extend(row[1:])

    # For y, take the mask image from index 0 and repeat it 25 times for each batch
    Y.extend([row[0]] * 25)

In [7]:
print(len(matrix))
print(len(X), len(Y))

400
10000 10000


In [8]:
import numpy as np
from PIL import Image


def resize_and_flatten_images(images, new_size):
    resized_images = [np.array(Image.fromarray(img).resize(new_size)) for img in images]
    
    # Ensure all images have the same shape
    common_shape = resized_images[0].shape
    resized_and_flattened_images = [img.flatten() / 255.0 for img in resized_images if img.shape == common_shape]
    
    return np.array(resized_and_flattened_images)

# # Example usage:
# # Assuming X and Y are lists of images, and you want to resize them to (128, 128)
# new_size = (128, 128)
# resized_and_flattened_X = resize_and_flatten_images(X, new_size)
# resized_and_flattened_Y = resize_and_flatten_images(Y, new_size)

In [9]:
def preprocess_data(X, Y, image_size=(128, 128)):
    # Resize and flatten images
    X = resize_and_flatten_images(X, image_size)
    Y = resize_and_flatten_images(Y, image_size)

    # Ensure X and Y have the same number of samples
    min_samples = min(len(X), len(Y))
    X = X[:min_samples]
    Y = Y[:min_samples]

    # Split the data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = preprocess_data(X, Y)

In [10]:
def create_unet_model(input_shape):
    inputs = keras.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    # Middle
    conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    # Decoder
    up4 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv3)
    concat4 = layers.Concatenate()([conv2, up4])
    conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat4)
    conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

    up5 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv4)
    concat5 = layers.Concatenate()([conv1, up5])
    conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(concat5)
    conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv5)

    model = models.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [11]:
def train_model(model, X_train, Y_train, epochs=16, batch_size=32):
    height, width = 128, 128  # Adjust based on your actual image size
    num_channels = 1  # Assuming grayscale images

    X_train_reshaped = X_train.reshape((-1, height, width, num_channels))
    Y_train_reshaped = Y_train.reshape((-1, height, width, num_channels))  # Reshape Y_train to match the model output shape

    model.fit(X_train_reshaped, Y_train_reshaped, epochs=epochs, batch_size=batch_size, validation_split=0.1)


In [12]:
height, width = 128, 128  # Adjust based on your actual image size
num_channels = 1  # Assuming grayscale images

X_train_reshaped = X_train.reshape((-1, height, width, num_channels))


print("Shape of X_train_reshaped:", X_train_reshaped.shape)
print("Shape of Y_train:", Y_train.shape)

Shape of X_train_reshaped: (6720, 128, 128, 1)
Shape of Y_train: (2240, 65536)


In [13]:
from tensorflow import keras
from tensorflow.keras import layers

input_shape = X_train_reshaped.shape[1:]
model = create_unet_model(input_shape)
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │     36,928 │ conv2d_2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    147,584 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 64, 64,    │     32,832 │ conv2d_5[0][0]    │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64, 64,    │          0 │ conv2d_3[0][0],   │
│ (Concatenate)       │ 128)              │            │ conv2d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 64, 64,    │     73,792 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 64, 64,    │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 128, 128,  │      8,224 │ conv2d_7[0][0]    │
│ (Conv2DTranspose)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 128, 128,  │          0 │ conv2d_1[0][0],   │
│ (Concatenate)       │ 64)               │            │ conv2d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 128, 128,  │     18,464 │ concatenate_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │      9,248 │ conv2d_8[0][0]  

 Total params: 465,953 (1.78 MB)

 Trainable params: 465,953 (1.78 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from keras import optimizers
from keras.optimizers import Adam


model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
epochs = 24
batch_size = 32
init_lr = 1e-4
optimizer = Adam(learning_rate=init_lr)


In [ ]:
# Train the model
train_model(model, X_train, Y_train)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Save the trained model
model.save("copy_move_detection_model.h5")

Epoch 1/16
166/189 ━━━━━━━━━━━━━━━━━━━━ 1:15 3s/step - accuracy: 0.9217 - loss: 0.2256